In [10]:
import os.path as osp
import os

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import negative_sampling

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])
path = osp.join(osp.dirname(osp.realpath(os.__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora', transform=transform)
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

In [14]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [15]:



model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6930, Val: 0.7362, Test: 0.7141
Epoch: 002, Loss: 0.6821, Val: 0.7325, Test: 0.7103
Epoch: 003, Loss: 0.7102, Val: 0.7257, Test: 0.7145
Epoch: 004, Loss: 0.6769, Val: 0.7289, Test: 0.7350
Epoch: 005, Loss: 0.6851, Val: 0.8040, Test: 0.8058
Epoch: 006, Loss: 0.6886, Val: 0.8361, Test: 0.8152
Epoch: 007, Loss: 0.6895, Val: 0.7904, Test: 0.7487
Epoch: 008, Loss: 0.6892, Val: 0.7575, Test: 0.7087
Epoch: 009, Loss: 0.6875, Val: 0.7489, Test: 0.6990
Epoch: 010, Loss: 0.6844, Val: 0.7517, Test: 0.7043
Epoch: 011, Loss: 0.6799, Val: 0.7583, Test: 0.7150
Epoch: 012, Loss: 0.6765, Val: 0.7645, Test: 0.7327
Epoch: 013, Loss: 0.6737, Val: 0.7588, Test: 0.7468
Epoch: 014, Loss: 0.6671, Val: 0.7491, Test: 0.7561
Epoch: 015, Loss: 0.6594, Val: 0.7387, Test: 0.7620
Epoch: 016, Loss: 0.6510, Val: 0.7315, Test: 0.7641
Epoch: 017, Loss: 0.6425, Val: 0.7314, Test: 0.7691
Epoch: 018, Loss: 0.6319, Val: 0.7429, Test: 0.7866
Epoch: 019, Loss: 0.6197, Val: 0.7754, Test: 0.8088
Epoch: 020, 